<a href="https://colab.research.google.com/github/Saifullah785/deep-learning-practice/blob/main/keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
import numpy as np
import pandas as pd

In [175]:
df = pd.read_csv('/content/drive/MyDrive/diabetes.csv')

In [176]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [177]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [178]:
x = df.iloc[:,0:-1].values
y = df.iloc[:,-1].values

In [179]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [180]:
x = scaler.fit_transform(x)

In [181]:
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [182]:
x.shape

(768, 8)

In [183]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1)

In [184]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [185]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [186]:
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4923 - loss: 0.7495 - val_accuracy: 0.5909 - val_loss: 0.6751
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6596 - loss: 0.6651 - val_accuracy: 0.6948 - val_loss: 0.6190
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6568 - loss: 0.6164 - val_accuracy: 0.7273 - val_loss: 0.5786
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6864 - loss: 0.5735 - val_accuracy: 0.7662 - val_loss: 0.5524
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7267 - loss: 0.5542 - val_accuracy: 0.7987 - val_loss: 0.5303
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7656 - loss: 0.5186 - val_accuracy: 0.7922 - val_loss: 0.5144
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7873 - loss: 0.4914 - val_accuracy: 0.7922 - val_loss: 0.5031
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7560 - loss: 0.5151 - val_accuracy: 0.7792 - 

In [187]:
# 1.how to select appropriate optimizer
# 2.No. of nodes in a layer
# 3.how to select no. of layers
# 4.All in all one model

In [188]:
# pip install -U keras-tuner

## 1.how to select appropriate optimizer

In [189]:

import keras_tuner as kt

In [190]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  optimizer=hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta' ])

  model.compile(optimizer= optimizer, loss= 'binary_crossentropy', metrics=['accuracy'])
  return model

In [191]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [192]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

In [193]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [194]:
model =tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [195]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [196]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6,validation_data=(x_test,y_test))


Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7385 - loss: 0.5280 - val_accuracy: 0.7727 - val_loss: 0.4911
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7154 - loss: 0.5169 - val_accuracy: 0.7727 - val_loss: 0.4835
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7525 - loss: 0.4846 - val_accuracy: 0.7857 - val_loss: 0.4784
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7296 - loss: 0.4972 - val_accuracy: 0.7727 - val_loss: 0.4744
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7806 - loss: 0.4622 - val_accuracy: 0.7727 - val_loss: 0.4706
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7773 - loss: 0.4685 - val_accuracy: 0.7727 - val_loss: 0.4695
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7866 - loss: 0.4708 - val_accuracy: 0.7922 - val_loss: 0.4681
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7905 - loss: 0.4398 - val_accuracy: 0.

## 2.No. of nodes in a layer

In [197]:
def build_model(hp):
  model = Sequential()


  units = hp.Int('units', min_value=8, max_value=128)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model



In [198]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='srk')

Reloading Tuner from mydir/srk/tuner0.json


In [199]:
tuner.search(x_train, y_train,epochs =5, validation_data=(x_test,y_test))


In [200]:
tuner.get_best_hyperparameters()[0].values

{'units': 105}

In [201]:
model = tuner.get_best_models(num_models=1)[0]

In [202]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7858 - loss: 0.4451 - val_accuracy: 0.8247 - val_loss: 0.4528
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7837 - loss: 0.4593 - val_accuracy: 0.8247 - val_loss: 0.4516
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7698 - loss: 0.4667 - val_accuracy: 0.8182 - val_loss: 0.4497
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7980 - loss: 0.4516 - val_accuracy: 0.8117 - val_loss: 0.4507
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7660 - loss: 0.4698 - val_accuracy: 0.8117 - val_loss: 0.4491
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7795 - loss: 0.4357 - val_accuracy: 0.8052 - val_loss: 0.4479
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7641 - loss: 0.4545 - val_accuracy: 0.8052 - val_loss: 0.4487
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7852 - loss: 0.4411 - val_accuracy: 0.81

## 3.how to select no. of layers

In [203]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu',input_dim=8))


  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(units=8, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer = 'rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [204]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='num_layers')

Reloading Tuner from mydir/num_layers/tuner0.json


In [205]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

In [206]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [207]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [208]:
model.fit(x_train, y_train, epochs=100, initial_epoch=6, validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7429 - loss: 0.6053 - val_accuracy: 0.7727 - val_loss: 0.5596
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7531 - loss: 0.5750 - val_accuracy: 0.7792 - val_loss: 0.5352
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7678 - loss: 0.5504 - val_accuracy: 0.7857 - val_loss: 0.5186
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8257 - loss: 0.4899 - val_accuracy: 0.7792 - val_loss: 0.5061
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7725 - loss: 0.5154 - val_accuracy: 0.7662 - val_loss: 0.4953
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7715 - loss: 0.5007 - val_accuracy: 0.7792 - val_loss: 0.4920
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7981 - loss: 0.4903 - val_accuracy: 0.7727 - val_loss: 0.4849
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7975 - loss: 0.4778 - val_accuracy: 0.

## 4.All in all one model

In [215]:
def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    if counter == 0:
      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=8, max_value=128,step=8),
              activation= hp.Choice('activation' + str(i) , values=['relu','tanh','sigmoid']),
              input_dim=8
              )
          )
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(
          Dense(
                hp.Int('units' + str(i), min_value=8, max_value=128,step=8),
                activation= hp.Choice('activation' + str(i) , values=['relu','tanh','sigmoid'])
                )
            )
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter += 1
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta' ]),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model


In [216]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=3,
                       directory='mydir',
                       project_name='final1')

Reloading Tuner from mydir/final1/tuner0.json


In [217]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

In [218]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 8,
 'activation0': 'tanh',
 'optimizer': 'adadelta',
 'units1': 64,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu',
 'units3': 48,
 'activation3': 'relu',
 'units4': 64,
 'activation4': 'sigmoid',
 'units5': 96,
 'activation5': 'tanh',
 'units6': 72,
 'activation6': 'tanh',
 'units7': 8,
 'activation7': 'tanh',
 'units8': 128,
 'activation8': 'relu',
 'units9': 88,
 'activation9': 'tanh'}

In [219]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adadelta', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [220]:
model.fit(x_train, y_train, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6692 - loss: 0.6417 - val_accuracy: 0.6429 - val_loss: 0.6592
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6488 - loss: 0.6657 - val_accuracy: 0.6429 - val_loss: 0.6590
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6562 - loss: 0.6557 - val_accuracy: 0.6429 - val_loss: 0.6588
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6404 - loss: 0.6759 - val_accuracy: 0.6429 - val_loss: 0.6586
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6355 - loss: 0.6751 - val_accuracy: 0.6429 - val_loss: 0.6584
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6521 - loss: 0.6598 - val_accuracy: 0.6429 - val_loss: 0.6582
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6514 - loss: 0.6608 - val_accuracy: 0.6429 - val_loss: 0.6579
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6600 - loss: 0.6472 - val_accuracy: 0.64